# 科学技術計算 4 レポート


In [14]:
import math
import cmath
import numpy as np
import scipy
from typing import Tuple
from tqdm.auto import tqdm

rng = np.random.default_rng()

## 課題04-1

In [ ]:
def lu_decomposition( A_org: np.ndarray ) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """LU decomposition of A = PLU

    Args:
        A (np.ndarray): nxn matrix A

    Returns:
        (np.ndarray): nxn permutation matrix P
        (np.ndarray): nxn lower matrix L with 1s in diagonal
        (np.ndarray): nxn strictly upper matrix U
    """
    assert A_org.ndim == 2
    assert A_org.shape[0] == A_org.shape[1]

    #前進消去を行うコード
    n = A_org.shape[0]
    A = A_org.copy()
    print("A\n", A, sep="")
    L = np.eye(n)   #単位行列生成
    P = np.eye(n)   #置換行列

    for j in range(n - 1):
        #ここから資料にあった部分ピポッド選択月も前進消去の実装
          
        #print(f"{j}-th row", A[j])
        
        # 部分ピボット選択
        j_max = np.abs(A[j:, j]).argmax() + j   #行中の一番大きい値を抽出
        if j != j_max:
            # A の行交換
            A[[j, j_max]] = A[[j_max, j]]

            # L の行交換（前の列のみ）
            if j > 0:
                L[[j, j_max], :j] = L[[j_max, j], :j]

            # P の行交換 ← これが scipy と同じ P を作るカギ
            P[[j, j_max]] = P[[j_max, j]]
        
        #print("A\n", A, sep="")
        #print("P\n", P)
    
        # 前進消去
        remaining_rows = n - (j + 1)
        r_ij = A[j + 1:, j] / A[j, j]
        A[j + 1:, j + 1:] -= np.tile(A[j, j + 1:], (remaining_rows, 1)) * r_ij.reshape(remaining_rows, -1)
        A[j + 1:, j] = 0
        L[j + 1:, j] = r_ij
        #print("A\n", A, sep="")
        #$print("L\n", L, sep="")
        
    U = A
    
    #print("U\n", U, sep="")

    P = P.T
    """
    #確認１
    print("P =",P)
    print("L =", L)
    print("U =", U)

    print("LU\n", L @ U, sep="")
    print("A\n", A_org, sep="")
    #print("LU == A", np.allclose(L @ U, A_org))
    """
    return P, L, U


for i in tqdm(range(50)):
    n = rng.integers(low = 2, high = 10) #サイズ2-10
    A = rng.random(size=(n, n))

    P, L, U = lu_decomposition(A)
    Psp, Lsp, Usp = scipy.linalg.lu(A)
    """
    print("Psp = ", Psp)
    print("Lsp = ", Lsp)
    print("Usp = ", Usp)
    """
 
    assert np.allclose(L, Lsp), "values don't match"
    assert np.allclose(U, Usp), "values don't match"
    
    assert np.allclose(P, Psp), "values don't match"

A
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]


  0%|          | 0/50 [00:00<?, ?it/s]

A
[[0.44268147 0.45153718 0.77923294]
 [0.35626905 0.22850192 0.38916992]
 [0.36878011 0.89930812 0.34061587]]
A
[[0.98978105 0.04162928 0.54981938 0.68269722 0.75900039 0.94768069]
 [0.96645901 0.99787817 0.36014302 0.3156221  0.11429498 0.2759584 ]
 [0.18326482 0.8098845  0.64654169 0.01498553 0.8545044  0.16636691]
 [0.85997697 0.87636003 0.56115556 0.19731735 0.29401514 0.4079664 ]
 [0.79644462 0.43537698 0.0300791  0.83931247 0.49412464 0.25604965]
 [0.45970778 0.53581694 0.78231672 0.32043383 0.28387035 0.1026921 ]]
A
[[0.32369909 0.73633691 0.38303823 0.96194628 0.37531527 0.61384652
  0.08437221 0.04264458]
 [0.22616115 0.48708315 0.10021103 0.9880134  0.47983294 0.78162691
  0.08369214 0.33734561]
 [0.12098904 0.42064823 0.48048808 0.14025106 0.6985283  0.34935886
  0.93840174 0.65101268]
 [0.37578538 0.26782721 0.05353903 0.25452234 0.71470874 0.6069299
  0.61889669 0.12035032]
 [0.31591586 0.36962413 0.84313813 0.50497366 0.98530517 0.79499874
  0.56464247 0.10697984]
 [0.89

## 課題04-2


In [16]:
def solve_lu( P: np.ndarray, L: np.ndarray, U: np.ndarray, b: np.ndarray,) -> np.ndarray:
    """Solve Ax = b with PLU, LU decomposition of A = PLU

    Args:
        P (np.ndarray): nxn permutation matrix P
        L (np.ndarray): nxn lower matrix L with 1s in diagonal
        U (np.ndarray): nxn strictly upper matrix U
        b (np.ndarray): n-d rhs vector b

    Returns:
        (np.ndarray): solution x of Ax = PLUx = b
    """
    assert b.ndim == 1
    n = b.shape[0]
    assert P.shape == (n, n)
    assert L.shape == (n, n)
    assert U.shape == (n, n)
    b = b.copy()

    # code here
    # solve Ly = P^T b
    a = P.T @ b
    # 前進代入
    y = np.zeros(n)
    for i in range(n):
        y[i] = a[i] - np.dot(L[i, :i], y[:i])
        # L[i,i] = 1（単位下三角行列）なので割り算不要

    # 後退代入
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]
    """
    x_true = solve(A, b)
    print("true solution x", x_true)
    print("x == x_true", np.allclose(x_true, x))
    print(x - x_true)
    """

    return x

for i in tqdm(range(50)):
    n = rng.integers(low=2, high=10)
    A = rng.random(size=(n, n))
    b = rng.random(n)

    P, L, U = lu_decomposition(A)
    x_with_lu = solve_lu(P, L, U, b) #こっちが自作

    x_numpy = np.linalg.solve(A, b) #こっちがライブラリにあるやつ

    assert np.allclose(x_numpy, x_with_lu ), "values don't match"

  0%|          | 0/50 [00:00<?, ?it/s]

A
[[0.36729083 0.91806419 0.00144443 0.06462806 0.08723621 0.08637044]
 [0.92951217 0.29706433 0.36277209 0.12301221 0.04207834 0.26358554]
 [0.10825497 0.70052371 0.05198755 0.4742789  0.94593301 0.60544848]
 [0.31049594 0.71195193 0.88150051 0.5307843  0.91157795 0.21413312]
 [0.20164308 0.41203424 0.187467   0.01777349 0.75420906 0.90899131]
 [0.84121642 0.24255061 0.99063271 0.75643214 0.78685475 0.52857516]]
A
[[0.35904426 0.00111137 0.45105275 0.12561588 0.41365    0.01024399
  0.26997644 0.30413059 0.00580764]
 [0.79509488 0.3872056  0.8721491  0.32860342 0.43791295 0.78445491
  0.26497618 0.35937704 0.69082277]
 [0.5427002  0.93191367 0.45112987 0.70062    0.0556701  0.57055753
  0.50773381 0.85647619 0.35966521]
 [0.57410453 0.39936354 0.14794863 0.15191146 0.04629663 0.79710625
  0.23016112 0.31559542 0.56738826]
 [0.88418146 0.93127764 0.66785198 0.39151115 0.67120305 0.88760485
  0.70066918 0.04646529 0.79360602]
 [0.36711328 0.56374595 0.63451544 0.94616475 0.91238675 0.61

## 課題04-4

In [145]:
#Pの行列式計算行う関数
def det_P(A: np.ndarray) -> float:
    
    """det(P) of permutation matrix P

    Args:
        P (np.ndarray): nxn permutation matrix of 0 and 1

    Returns:
        float: det(P) that is either 1 or -1
    """

    assert P.ndim == 2
    assert P.shape[0] == P.shape[1]

    for p in P.ravel()[np.nonzero(P.ravel())]:
        assert np.isclose(p, 1.0)
    
    n = P.shape[0]
    assert np.allclose(P.sum(axis=0), np.ones(n))
    assert np.allclose(P.sum(axis=1), np.ones(n))

    ## below three lines of code are helped with ChatGPT-4o because of its complexisity
    
    # Extract the permutation using np.nonzero
    permutation = np.nonzero(P)[1]

    # Calculate the number of inversions in the permutation
    inversions = sum(1 for i in range(len(permutation)) for j in range(i + 1, len(permutation)) if permutation[i] > permutation[j])

    # Return 1 if inversions are even, otherwise -1
    return 1 if inversions % 2 == 0 else -1


#Aの行列式をPLU分解を用いて行列式の計算行う関数
def det_lu( A_org: np.ndarray,) -> float:
    """compute det(A)

    Args:
        A (np.ndarray): nxn square matrix

    Returns:
        (float): det(A)
    """
    A = A_org.copy()
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    
    P, L, U = lu_decomposition(A)
    # code here
    #U求めちゃってかければハッピー(Lはスカラーだからたぶんどうにかなる)
    signP = det_P(P)   #Pの行列式の計算

    # U の対角成分の積（logで安全に計算
    diagU = np.diag(U) #対角成分だけとるここまであっている

    # 符号の計算
    log_symbol_U = np.prod(np.sign(diagU))
    

    # 対角絶対値の log の和
    log_abs_U = np.sum(np.log(np.abs(diagU))) #logをとる

    signU = log_symbol_U * np.exp(log_abs_U)
    detA = signP * signU

    return signU



#実行
for i in tqdm(range(10)):
    n = rng.integers(low=2, high=5)
    B = rng.random(size=(n, n))
    
    #行列式計算(自作関数)
    det_A = det_lu(A)
    #numpyのやつ
    A_numpy_det = np.linalg.det(A)
    

    print(det_A)
    print(A_numpy_det)

    assert np.isclose( det_A, A_numpy_det), "values don't match"


  0%|          | 0/10 [00:00<?, ?it/s]

A
[[0.76891657 0.91894134 0.67543546]
 [0.63489377 0.02378371 0.19674228]
 [0.33557979 0.90095068 0.38192619]]
0.08949754247383038
0.08949754247383034
A
[[0.76891657 0.91894134 0.67543546]
 [0.63489377 0.02378371 0.19674228]
 [0.33557979 0.90095068 0.38192619]]
0.08949754247383038
0.08949754247383034
A
[[0.76891657 0.91894134 0.67543546]
 [0.63489377 0.02378371 0.19674228]
 [0.33557979 0.90095068 0.38192619]]
0.08949754247383038
0.08949754247383034
A
[[0.76891657 0.91894134 0.67543546]
 [0.63489377 0.02378371 0.19674228]
 [0.33557979 0.90095068 0.38192619]]
0.08949754247383038
0.08949754247383034
A
[[0.76891657 0.91894134 0.67543546]
 [0.63489377 0.02378371 0.19674228]
 [0.33557979 0.90095068 0.38192619]]
0.08949754247383038
0.08949754247383034
A
[[0.76891657 0.91894134 0.67543546]
 [0.63489377 0.02378371 0.19674228]
 [0.33557979 0.90095068 0.38192619]]
0.08949754247383038
0.08949754247383034
A
[[0.76891657 0.91894134 0.67543546]
 [0.63489377 0.02378371 0.19674228]
 [0.33557979 0.90095

## 課題04-7